# Quantum ESPRESSO Workshop - Part 4: K-point Convergence

## Learning Objectives
1. Understand Brillouin zone sampling and k-points
2. Perform systematic k-point convergence tests
3. Determine the converged k-point mesh
4. Understand symmetry reduction of k-points

---

## 1. Why K-point Convergence Matters

### Brillouin Zone Integration

Many physical quantities require integration over the Brillouin zone:

$$\bar{f} = \frac{1}{V_{BZ}} \int_{BZ} f(\mathbf{k}) d\mathbf{k}$$

In practice, this integral is approximated by a discrete sum:

$$\bar{f} \approx \sum_i w_i f(\mathbf{k}_i)$$

where $w_i$ are weights and $\mathbf{k}_i$ are sampling points.

### Monkhorst-Pack Grid

The most common method is the **Monkhorst-Pack** (MP) grid:
- Uniform grid of $n_1 \times n_2 \times n_3$ points
- Can be shifted (offset) from Γ point
- Symmetry operations reduce the number of unique k-points

### Convergence Criterion

Like ecutwfc, we need to ensure the k-point mesh is dense enough:
- Too sparse: Poor sampling → inaccurate results
- Too dense: Unnecessary computational cost
- Goal: Converged total energy to within 1 meV/atom

---

## 2. Setup

In [ ]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import re
import time
import json

# Paths
WORKSHOP_ROOT = Path('/home/claude/qe_workshop')
PSEUDO_DIR = WORKSHOP_ROOT / 'pseudopotentials'
OUTPUT_DIR = WORKSHOP_ROOT / 'outputs'

# Working directory
WORK_DIR = OUTPUT_DIR / '04_kpoint_convergence'
WORK_DIR.mkdir(exist_ok=True)
(WORK_DIR / 'tmp').mkdir(exist_ok=True)

print(f"Working directory: {WORK_DIR}")

In [ ]:
# Load converged ecutwfc from previous notebook
params_file = WORKSHOP_ROOT / 'converged_parameters.json'

if params_file.exists():
    with open(params_file, 'r') as f:
        converged_params = json.load(f)
    ecutwfc = converged_params['ecutwfc_recommended']
    ecutrho_factor = converged_params['ecutrho_factor']
    print(f"Loaded converged parameters:")
    print(f"  ecutwfc = {ecutwfc} Ry (recommended)")
    print(f"  ecutrho = {ecutrho_factor} × ecutwfc")
else:
    # Default values if previous notebook not run
    ecutwfc = 40.0
    ecutrho_factor = 8
    print(f"Using default parameters (run Notebook 03 first for converged values):")
    print(f"  ecutwfc = {ecutwfc} Ry")
    print(f"  ecutrho = {ecutrho_factor} × ecutwfc")

---

## 3. Helper Functions

In [ ]:
def generate_scf_input(prefix, ecutwfc, ecutrho, kpoints, pseudo_dir,
                       celldm1=10.26, conv_thr=1.0e-8):
    """Generate SCF input file for Silicon."""
    kx, ky, kz = kpoints
    
    input_text = f"""&CONTROL
    calculation = 'scf'
    prefix = '{prefix}'
    outdir = './tmp'
    pseudo_dir = '{pseudo_dir}'
    verbosity = 'high'
    tprnfor = .true.
    tstress = .true.
/

&SYSTEM
    ibrav = 2
    celldm(1) = {celldm1}
    nat = 2
    ntyp = 1
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'smearing'
    smearing = 'cold'
    degauss = 0.01
/

&ELECTRONS
    conv_thr = {conv_thr}
    mixing_beta = 0.7
/

ATOMIC_SPECIES
    Si  28.0855  Si.upf

ATOMIC_POSITIONS {{crystal}}
    Si  0.00  0.00  0.00
    Si  0.25  0.25  0.25

K_POINTS {{automatic}}
    {kx} {ky} {kz} 0 0 0
"""
    return input_text


def parse_output(output_text):
    """Parse total energy and number of k-points from output."""
    results = {}
    
    for line in output_text.split('\n'):
        if '!' in line and 'total energy' in line:
            match = re.search(r'=\s+([\d.E+-]+)\s+Ry', line)
            if match:
                results['energy_ry'] = float(match.group(1))
        
        if 'number of k points' in line:
            match = re.search(r'=\s+(\d+)', line)
            if match:
                results['nkpts_irreducible'] = int(match.group(1))
    
    results['converged'] = 'convergence has been achieved' in output_text
    
    return results


def run_pwscf(input_file, timeout=300):
    """Run pw.x calculation."""
    input_file = Path(input_file)
    output_file = input_file.with_suffix('.out')
    work_dir = input_file.parent
    
    start = time.time()
    result = subprocess.run(
        ['pw.x', '-in', input_file.name],
        capture_output=True,
        text=True,
        cwd=work_dir,
        timeout=timeout
    )
    elapsed = time.time() - start
    
    with open(output_file, 'w') as f:
        f.write(result.stdout)
    
    return result.stdout, elapsed

---

## 4. K-point Convergence Test

For cubic systems like Si, we use uniform grids: $n \times n \times n$

We'll test grids from 2×2×2 to 14×14×14.

In [ ]:
# Define test parameters
kpoint_grids = [2, 4, 6, 8, 10, 12, 14]  # n×n×n grids
ecutrho = ecutwfc * ecutrho_factor

print("K-point Convergence Test Parameters")
print("=" * 50)
print(f"K-point grids: {' × '.join([f'{k}×{k}×{k}' for k in kpoint_grids])}")
print(f"ecutwfc = {ecutwfc} Ry (converged)")
print(f"ecutrho = {ecutrho} Ry")
print(f"Lattice parameter: 10.26 Bohr (experimental)")

In [ ]:
# Run convergence tests
results = []

print("\nRunning k-point convergence tests...")
print("=" * 80)
print(f"{'K-grid':<12} {'Total k-pts':<12} {'Irred. k-pts':<15} {'Energy (Ry)':<20} {'Time (s)':<10} {'Status'}")
print("-" * 80)

for k in kpoint_grids:
    kpoints = (k, k, k)
    total_kpts = k * k * k
    prefix = f'si_k{k}'
    
    # Generate input
    input_text = generate_scf_input(
        prefix=prefix,
        ecutwfc=ecutwfc,
        ecutrho=ecutrho,
        kpoints=kpoints,
        pseudo_dir=PSEUDO_DIR,
        conv_thr=1.0e-8
    )
    
    # Write and run
    input_file = WORK_DIR / f'{prefix}.in'
    with open(input_file, 'w') as f:
        f.write(input_text)
    
    output, elapsed = run_pwscf(input_file)
    parsed = parse_output(output)
    
    status = '✓' if parsed.get('converged', False) else '✗'
    energy_str = f"{parsed.get('energy_ry', 0):.8f}" if parsed.get('energy_ry') else 'N/A'
    nkpts_irr = parsed.get('nkpts_irreducible', 'N/A')
    
    print(f"{k}×{k}×{k:<8} {total_kpts:<12} {nkpts_irr:<15} {energy_str:<20} {elapsed:<10.2f} {status}")
    
    results.append({
        'kgrid': k,
        'total_kpts': total_kpts,
        'irreducible_kpts': nkpts_irr if isinstance(nkpts_irr, int) else None,
        'energy_ry': parsed.get('energy_ry'),
        'time_s': elapsed,
        'converged': parsed.get('converged', False)
    })

print("=" * 80)
print("K-point convergence tests complete!")

In [ ]:
# Save results
results_file = WORK_DIR / 'kpoint_convergence_results.json'
with open(results_file, 'w') as f:
    json.dump(results, f, indent=2)
print(f"Results saved to: {results_file}")

---

## 5. Analyze Convergence

In [ ]:
# Extract converged results
converged_results = [r for r in results if r['converged'] and r['energy_ry'] is not None]

if len(converged_results) < 2:
    print("ERROR: Not enough converged calculations!")
else:
    kgrid_arr = np.array([r['kgrid'] for r in converged_results])
    energy_arr = np.array([r['energy_ry'] for r in converged_results])
    time_arr = np.array([r['time_s'] for r in converged_results])
    nkpts_arr = np.array([r['irreducible_kpts'] for r in converged_results])
    
    # Reference: densest grid
    E_ref = energy_arr[-1]
    delta_E_meV = (energy_arr - E_ref) * 13605.693 / 2  # meV per atom
    
    print("K-point Convergence Analysis")
    print("=" * 90)
    print(f"Reference: {kgrid_arr[-1]}×{kgrid_arr[-1]}×{kgrid_arr[-1]} grid, E = {E_ref:.8f} Ry")
    print(f"\n{'K-grid':<12} {'Irred. k-pts':<15} {'Energy (Ry)':<18} {'ΔE (meV/atom)':<18} {'Time (s)'}")
    print("-" * 90)
    
    for i in range(len(converged_results)):
        k = kgrid_arr[i]
        print(f"{k}×{k}×{k:<8} {nkpts_arr[i]:<15} {energy_arr[i]:<18.8f} {delta_E_meV[i]:<18.4f} {time_arr[i]:.2f}")
    
    print("=" * 90)

In [ ]:
# Determine converged k-point grid
convergence_threshold_meV = 1.0

print(f"\nConvergence criterion: |ΔE| < {convergence_threshold_meV} meV/atom")
print("-" * 50)

converged_kgrid = None
for i in range(len(converged_results) - 1):
    if abs(delta_E_meV[i]) < convergence_threshold_meV:
        converged_kgrid = kgrid_arr[i]
        print(f"\n✓ Converged k-grid: {converged_kgrid}×{converged_kgrid}×{converged_kgrid}")
        print(f"  Irreducible k-points: {nkpts_arr[i]}")
        print(f"  ΔE = {delta_E_meV[i]:.4f} meV/atom")
        break

if converged_kgrid is None:
    print("\n⚠ No k-grid satisfies the convergence criterion!")
    print("  Consider testing denser grids.")

---

## 6. Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# X-axis labels
kgrid_labels = [f'{k}×{k}×{k}' for k in kgrid_arr]
x_pos = np.arange(len(kgrid_arr))

# Plot 1: Energy vs k-grid
ax1 = axes[0, 0]
ax1.plot(x_pos, energy_arr, 'bo-', markersize=10, linewidth=2)
ax1.set_xticks(x_pos)
ax1.set_xticklabels(kgrid_labels, rotation=45)
ax1.set_xlabel('K-point Grid', fontsize=12)
ax1.set_ylabel('Total Energy (Ry)', fontsize=12)
ax1.set_title('Total Energy vs K-point Sampling', fontsize=14)
ax1.grid(True, alpha=0.3)

# Plot 2: Energy convergence
ax2 = axes[0, 1]
ax2.semilogy(x_pos[:-1], np.abs(delta_E_meV[:-1]), 'ro-', markersize=10, linewidth=2)
ax2.axhline(y=convergence_threshold_meV, color='g', linestyle='--',
            label=f'Threshold = {convergence_threshold_meV} meV/atom')
ax2.set_xticks(x_pos[:-1])
ax2.set_xticklabels(kgrid_labels[:-1], rotation=45)
ax2.set_xlabel('K-point Grid', fontsize=12)
ax2.set_ylabel('|ΔE| (meV/atom)', fontsize=12)
ax2.set_title('K-point Convergence', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Time vs k-grid
ax3 = axes[1, 0]
ax3.bar(x_pos, time_arr, color='steelblue', alpha=0.7)
ax3.set_xticks(x_pos)
ax3.set_xticklabels(kgrid_labels, rotation=45)
ax3.set_xlabel('K-point Grid', fontsize=12)
ax3.set_ylabel('Wall Time (s)', fontsize=12)
ax3.set_title('Computational Cost vs K-points', fontsize=14)
ax3.grid(True, alpha=0.3, axis='y')

# Plot 4: Number of irreducible k-points
ax4 = axes[1, 1]
total_kpts = kgrid_arr ** 3
ax4.bar(x_pos - 0.15, total_kpts, width=0.3, label='Total k-points', color='lightcoral', alpha=0.7)
ax4.bar(x_pos + 0.15, nkpts_arr, width=0.3, label='Irreducible k-points', color='steelblue', alpha=0.7)
ax4.set_xticks(x_pos)
ax4.set_xticklabels(kgrid_labels, rotation=45)
ax4.set_xlabel('K-point Grid', fontsize=12)
ax4.set_ylabel('Number of K-points', fontsize=12)
ax4.set_title('Symmetry Reduction of K-points', fontsize=14)
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(str(WORK_DIR / 'kpoint_convergence.png'), dpi=150, bbox_inches='tight')
plt.show()
print(f"\nFigure saved to: {WORK_DIR / 'kpoint_convergence.png'}")

---

## 7. Understanding Symmetry Reduction

Notice that the number of **irreducible k-points** is much smaller than the total grid size. This is because:

1. **Point group symmetry**: Si (diamond structure) has 48 point group operations (Oh + inversion)
2. **Time-reversal symmetry**: $E(\mathbf{k}) = E(-\mathbf{k})$

QE automatically applies these symmetries to reduce computational cost.

In [ ]:
# Calculate symmetry reduction ratio
print("Symmetry Reduction Analysis")
print("=" * 60)
print(f"{'K-grid':<12} {'Total k-pts':<15} {'Irreducible':<15} {'Reduction'}")
print("-" * 60)

for i in range(len(converged_results)):
    k = kgrid_arr[i]
    total = k ** 3
    irr = nkpts_arr[i]
    reduction = total / irr if irr > 0 else 0
    print(f"{k}×{k}×{k:<8} {total:<15} {irr:<15} {reduction:.1f}×")

print("=" * 60)
print("\nNote: Higher reduction ratios mean more efficient use of symmetry.")

---

## 8. Update Converged Parameters

In [ ]:
# Update converged parameters file
params_file = WORKSHOP_ROOT / 'converged_parameters.json'

if params_file.exists():
    with open(params_file, 'r') as f:
        params = json.load(f)
else:
    params = {}

# Add k-point convergence results
params['kpoints_converged'] = int(converged_kgrid) if converged_kgrid else 8
params['kpoints_recommended'] = int(converged_kgrid) + 2 if converged_kgrid else 10  # Safety margin
params['kpoint_convergence_criterion_meV'] = convergence_threshold_meV

with open(params_file, 'w') as f:
    json.dump(params, f, indent=2)

print("Updated converged parameters:")
print(json.dumps(params, indent=2))
print(f"\nSaved to: {params_file}")

---

## 9. Practical Guidelines for K-point Selection

### General Rules

| System Type | Recommended K-point Density |
|------------|-----------------------------|
| Bulk metals | Dense (10-20 per Å⁻¹) |
| Semiconductors | Moderate (6-10 per Å⁻¹) |
| Insulators | Sparse (4-6 per Å⁻¹) |
| Molecules/surfaces | Gamma only or sparse |

### K-point Density Formula

A useful rule of thumb:

$$n = \text{round}\left(\frac{R \cdot a}{2\pi}\right)$$

where $R$ is the desired k-point density (typically 30-50 for semiconductors) and $a$ is the lattice parameter.

### For Band Structure Calculations

Use much denser k-points along high-symmetry paths (typically 20-50 points between special points).

In [ ]:
# Calculate k-point density
a_bohr = 10.26  # Lattice parameter in Bohr
a_angstrom = a_bohr * 0.529177

print("K-point Density Analysis")
print("=" * 60)
print(f"Lattice parameter: a = {a_bohr} Bohr = {a_angstrom:.3f} Å")
print(f"\n{'K-grid':<12} {'K-density (Å⁻¹)':<20} {'K-density (Bohr⁻¹)'}")
print("-" * 60)

for k in kgrid_arr:
    # K-spacing in reciprocal space
    k_density_angstrom = k / a_angstrom
    k_density_bohr = k / a_bohr
    print(f"{k}×{k}×{k:<8} {k_density_angstrom:<20.3f} {k_density_bohr:.3f}")

print("=" * 60)

---

## Summary

In this notebook, we have:

1. ✓ Understood Brillouin zone sampling
2. ✓ Performed k-point convergence tests (2×2×2 to 14×14×14)
3. ✓ Determined the converged k-point mesh
4. ✓ Analyzed symmetry reduction
5. ✓ Updated converged parameters

### Key Findings for Silicon

- **Converged k-grid** (1 meV/atom): {converged_kgrid}×{converged_kgrid}×{converged_kgrid} (if determined)
- **Recommended k-grid** (with safety): ({converged_kgrid+2} if converged_kgrid else 10)×({converged_kgrid+2} if converged_kgrid else 10)×({converged_kgrid+2} if converged_kgrid else 10)

### Combined Converged Parameters (so far)

- ecutwfc: from Notebook 03
- k-points: from this notebook
- Next: Lattice parameter optimization

### Next Notebook
→ **05_Lattice_Optimization.ipynb**: Finding the equilibrium lattice parameter